In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-04-13 13:15:01,666 : INFO : 'pattern' package not found; tag filters are not available for English


In [6]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [7]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield text
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield text
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield text
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield text
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield text
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield text
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield text
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield text
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [8]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-04-13 13:15:28,258 : INFO : loading Dictionary object from stem100_dictionary
2018-04-13 13:15:28,299 : INFO : loaded stem100_dictionary


In [9]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-04-13 13:15:28,611 : INFO : loaded corpus index from stem100.mm.index
2018-04-13 13:15:28,619 : INFO : initializing cython corpus reader from stem100.mm
2018-04-13 13:15:28,620 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [10]:
# %%time

# c = texts()
# count = 0
# for x in c.everything():
#     x = [ i for i in x if i in dictionary.token2id]
#     with open(os.path.normpath('Token_corpus/' + str(count)), 'wb') as fp:
#         pickle.dump(x, fp)
#     count=count+1


In [11]:
def token_corpus(i):
    with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
            itemlist = pickle.load(fp)
            return itemlist   

In [12]:
class iter_token_corpus:
    def __iter__(self):
        for i in xrange(0, 351985):
            with open (os.path.normpath('Token_corpus/' + str(i)), 'rb') as fp:
                itemlist = pickle.load(fp)
                yield itemlist

In [13]:
# %%time
# model = gensim.models.Word2Vec(iter_token_corpus(), size=300, iter = 10, min_count=0, sample=0, workers=8)

In [14]:
# model.save('word2vec')

In [16]:
len(corpus)

351985

In [15]:
model = gensim.models.Word2Vec.load('word2vecsubsam')

2018-04-13 13:15:29,348 : INFO : loading Word2Vec object from word2vecsubsam
2018-04-13 13:15:29,755 : INFO : loading wv recursively from word2vecsubsam.wv.* with mmap=None
2018-04-13 13:15:29,756 : INFO : loading syn0 from word2vecsubsam.wv.syn0.npy with mmap=None
2018-04-13 13:15:30,162 : INFO : setting ignored attribute syn0norm to None
2018-04-13 13:15:30,164 : INFO : loading syn1neg from word2vecsubsam.syn1neg.npy with mmap=None
2018-04-13 13:15:30,553 : INFO : Model saved using code from earlier Gensim Version. Re-loading old model in a compatible way.
2018-04-13 13:15:30,554 : INFO : loading Word2Vec object from word2vecsubsam
2018-04-13 13:15:30,882 : INFO : loading wv recursively from word2vecsubsam.wv.* with mmap=None
2018-04-13 13:15:30,883 : INFO : loading syn0 from word2vecsubsam.wv.syn0.npy with mmap=None
2018-04-13 13:15:30,907 : INFO : setting ignored attribute syn0norm to None
2018-04-13 13:15:30,908 : INFO : loading syn1neg from word2vecsubsam.syn1neg.npy with mmap=No

In [13]:
# x.doesnt_match(words)

# #     Which word from the given list doesn’t go with the others?
# #     Parameters:	words – List of words
# #     Returns:	The word further away from the mean of all words.
# #     Return type:	str

# x.index2word
# ### list of words in the sorted order of their frequency 

# y = x.vocab['parti']
# y.count
# # find freq. of any word


# x.most_similar_cosmul(positive=None, negative=None, topn=10)
# # Find the top-N most similar words, using the multiplicative combination objective proposed by 
# # Omer Levy and Yoav Goldberg.Positive words still contribute positively towards the similarity, negative words 
# # negatively, but with less susceptibility to one large distance dominating the calculation.

# x.most_similar_to_given(entity1, entities_list)
# #    Return the entity from entities_list most similar to entity1.

# x.n_similarity(ws1, ws2)
# #Compute cosine similarity between two sets of words


# x.similar_by_vector(x['fit'], topn=10, restrict_vocab=None)
# # closest vectors
# # if topn is False, similar_by_vector returns the vector of similarity scores.

# x.similar_by_word(word, topn=10, restrict_vocab=None)
# #  Find the top-N most similar words.

# x.similarity(w1, w2)
# #Compute cosine similarity between two words.

# x.wmdistance(doc1, doc2)
# # word movers distance

# y = x.vocab['parti']
# y.count
# #freq in corpus

In [14]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
distance_cache = {}


In [96]:
from pyemd import emd

from gensim.corpora.dictionary import Dictionary

from numpy import dot, float32 as REAL, empty, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
ndarray, sum as np_sum, prod, argmax, divide as np_divide


w2v  = model.wv
def wmdistance(document1, document2, dlen1, dlen2):
    # Sets for faster look-up.
    docset1 = set([i[0] for i in document1])
    docset2 = set([i[0] for i in document2])
    voclist = sorted(set.union(docset1, docset2))
    vocab_len = len(voclist)
    voc_dic = {k: v for v, k in enumerate(voclist)}
    
    
    # Compute distance matrix.
    distance_matrix = zeros((vocab_len, vocab_len), dtype=double)
    for i in docset1:
        for j in docset2:
            if (i,j) not in distance_cache:    
                # Compute Euclidean distance between word vectors.
                distance_matrix[voc_dic[i], voc_dic[j]] = sqrt(np_sum((w2v[dictionary.id2token[i]] - w2v[dictionary.id2token[j]])**2))
                distance_cache[(i,j)] = distance_matrix[voc_dic[i], voc_dic[j]]
            else:
                distance_matrix[voc_dic[i], voc_dic[j]] = distance_cache[(i,j)]
    
    #print distance_matrix
    if np_sum(distance_matrix) == 0.0:
        # `emd` gets stuck if the distance matrix contains only zeros.
        return -1000

    d1 = zeros(vocab_len, dtype=double)
    d2 = zeros(vocab_len, dtype=double)
    for w, freq in document1:
        d1[voc_dic[w]] = freq / float(dlen1)  # Normalized word frequencies.
    for w, freq in document2:
        d2[voc_dic[w]] = freq / float(dlen2)  # Normalized word frequencies.

    # Compute WMD.
    return emd(d1, d2, distance_matrix)

In [2]:
len(corpus)

NameError: name 'corpus' is not defined

In [90]:
# load doc_len
with open('doc_len_stem100.pkl', 'rb') as ip:
    doc_len = pickle.load(ip)

In [ ]:
docsets = []
for x in corpus:
    docsets.append(set([i[0] for i in x]))



In [1]:
%%time
results = copy.deepcopy(query_truth)
sim_list = [0] + [[] for i in range(10)]
rank_list = [0]
queries = [0]
qlen = [0]
qbow = [0]
qsets = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    query = [i for i in query if i in dictionary.token2id]
    qlen.append(len(query))
    queries.append(query)
    qbow.append(dictionary.doc2bow(query)
    qsets.append( set([i[0] for i in query]) )
t = 0
for d,doc in enumerate(corpus):
    if doc:
        for i in range(1, 11):
            docset1 = qsets[i]
            docset2 = docsets[d]
            dlen1 = qlen[i]
            dlen2 = doc_len[d]
            voclist = sorted(set.union(docset1, docset2))
            vocab_len = len(voclist)
            voc_dic = {k: v for v, k in enumerate(voclist)}
          # Compute distance matrix.
            distance_matrix = zeros((vocab_len, vocab_len), dtype=double)
            for i in docset1:
                for j in docset2:
                    if (i,j) not in distance_cache:    
                        # Compute Euclidean distance between word vectors.
                        distance_matrix[voc_dic[i], voc_dic[j]] = sqrt(np_sum((w2v[dictionary.id2token[i]] - w2v[dictionary.id2token[j]])**2))
                        distance_cache[(i,j)] = distance_matrix[voc_dic[i], voc_dic[j]]
                    else:
                        distance_matrix[voc_dic[i], voc_dic[j]] = distance_cache[(i,j)]

            #print distance_matrix
            if np_sum(distance_matrix) == 0.0:
                # `emd` gets stuck if the distance matrix contains only zeros.
                return -1000

            d1 = zeros(vocab_len, dtype=double)
            d2 = zeros(vocab_len, dtype=double)
            for w, freq in document1:
                d1[voc_dic[w]] = freq / float(dlen1)  # Normalized word frequencies.
            for w, freq in document2:
                d2[voc_dic[w]] = freq / float(dlen2)  # Normalized word frequencies.

            # Compute WMD.
            sim_list[i].append(emd(d1, d2, distance_matrix))
     else:
        for i in range(1, 11):
            sim_list[i].append(-1000)
    t = t + 1
    #print t
    if t >500:
        break
for i in range(1, 11):
    break
    ranks = rankdata(sims[i], method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])

        

SyntaxError: invalid syntax (<unknown>, line 13)

In [30]:
sim_list

[0,
 [16.890827099511526],
 [15.420610249286486],
 [16.063524800096037],
 [14.813866890527427],
 [15.612927507185416],
 [17.153650218064293],
 [15.523167647444359],
 [15.43141604724848],
 [15.142004262530309],
 [16.1761308301454]]

In [ ]:
results